Дана статистика пользователей adult.csv.

Получите значения AUC для различных моделей и их параметров.

Решением будет ссылка на гитхаб с ноутбуком.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling

In [2]:
data = pd.read_csv("adult.csv")

In [3]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [4]:
data.shape

(48842, 15)

In [18]:
pandas_profiling.ProfileReport(data)

In [3]:
# сделаем первичную подготовку данных
# удаляем имеющиеся дубликаты

data = data.drop_duplicates()

In [73]:
data.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,48790.000000,4.879000e+04,48790.000000,48790.000000,48790.000000,48790.000000
mean,38.652798,1.896690e+05,10.078807,1080.217688,87.595573,40.425886
std,13.708493,1.056172e+05,2.570046,7455.905921,403.209129,12.392729
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175550e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781385e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376062e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [4]:
# удаляю часть признаков 'fnlwgt' не понятный признак. 'capital-gain' 'capital-loss' имеют более 90% нулевые значения

data = data.drop(['fnlwgt', 'capital-gain', 'capital-loss'], axis=1)

In [5]:
# целевой признак переведем в бинарные значения

data = data.replace({'income': {'<=50K': 0, '>50K': 1}})

In [6]:
# преобразуем признаки

data = pd.get_dummies(data, drop_first=True)

In [139]:
data.head()

,age,workclass,education,educational-num,marital-status,occupation,relationship,race,gender,hours-per-week,native-country,income
0,25,4,1,7,4,7,3,2,1,40,39,0
1,38,4,11,9,2,5,0,4,1,50,39,0
2,28,2,7,12,2,11,0,4,1,40,39,1
3,44,4,15,10,2,7,0,2,1,40,39,1
4,18,0,15,10,4,0,3,4,0,30,39,0


In [7]:
# делим данные на X, y

X = data.drop('income', axis=1)
y = data['income']

In [8]:
# Поделим на train и test

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score

In [34]:
# определяем модели которые будут учавствовать в обучениях

estimators = [('linear', LinearRegression()), ('svr', SVR()), ('trees', DecisionTreeRegressor()), 
              ('forest', RandomForestRegressor())]

In [36]:
for model in estimators:
    model[1].fit(X_train, y_train)
    predictions = model[1].predict(X_test)
    print(f'Качество модели {model[0]}: {roc_auc_score(y_test, predictions):.2f}')

Качество модели linear: 0.82
Качество модели svr: 0.83
Качество модели trees: 0.73
Качество модели forest: 0.85


Наилучший результат по показателю roc_auc_score показала модель Случайный лес

In [ ]:
# ПРОБЛЕМА
# проверяю качестов модели на кросс валидации и получаю ужасный результат, но не пойму с чем это связано?

In [41]:
clf = RandomForestRegressor(n_estimators=10)
clf = clf.fit(X_train, y_train)

In [42]:
scores = cross_val_score(clf, X_train, y_train, cv=10)
print(f'Оценка качества модели: {scores.mean():.2f}')

Оценка качества модели: 0.24
